In [ ]:
import os
from PIL import Image
import numpy as np
import fiftyone as fo
import fiftyone.zoo as foz

# ---- BAGIAN 1: HAPUS DATASET LAMA JIKA ADA ----
dataset_name = "my-fish-dataset"
# Cek apakah dataset dengan nama tersebut sudah ada
if dataset_name in fo.list_datasets():
    print(f"⚠️  Dataset '{dataset_name}' sudah ada. Menghapus yang lama...")
    fo.delete_dataset(dataset_name)
    print(f"✅ Dataset lama berhasil dihapus.")

# ---- BAGIAN 2: MUAT DATASET DENGAN CARA YANG SUDAH BERHASIL ----
print("\n⏳ Memuat dataset Open Images V7 (subset ikan)...")
# GUNAKAN KODE YANG SUDAH BERHASIL SEBELUMNYA
dataset = foz.load_zoo_dataset(
    "open-images-v7",
    split="validation",           # Gunakan 'validation' sesuai folder Anda
    label_types=["detections"],
    classes=["Fish"],
    max_samples=5000,             # Anda sudah set ke 5000 sebelumnya
    # JANGAN gunakan parameter 'dataset_dir' di sini
    shuffle=True,
    dataset_name=dataset_name     # Beri nama yang sama
)

print(f"✅ Dataset berhasil dimuat!")
print(f"   Nama: {dataset.name}")
print(f"   Jumlah Sampel Gambar: {len(dataset)}")

# Verifikasi cepat
if len(dataset) > 0:
    sample = dataset.first()
    print(f"\n🔍 Contoh Sampel Pertama:")
    print(f"   - Path File: {sample.filepath}")
    print(f"   - File ada? : {os.path.exists(sample.filepath)}")
    if hasattr(sample, 'detections') and sample.detections:
        print(f"   - Jumlah Deteksi 'Fish': {len(sample.detections.detections)}")
        if len(sample.detections.detections) > 0:
            det = sample.detections.detections[0]
            print(f"     * Bounding Box: {det.bounding_box}")
            # Buka gambar dan tampilkan ukurannya
            try:
                img = Image.open(sample.filepath)
                print(f"     * Ukuran Gambar Asli: {img.size}")
                img.close()
            except:
                print("     * (Tidak bisa membuka gambar)")

# ---- BAGIAN 3: PROSES CROPPING IKAN ----
print("\n" + "="*60)
print("MEMULAI PROSES CROPPING GAMBAR IKAN...")
print("="*60)

# 1. Tentukan struktur folder tujuan
base_crop_dir = "./fish_crops"  # Folder utama untuk gambar hasil crop
train_dir = os.path.join(base_crop_dir, "train")  # Subfolder untuk data latih
val_dir = os.path.join(base_crop_dir, "val")      # Subfolder untuk data validasi

# Buat folder jika belum ada
os.makedirs(train_dir, exist_ok=True)
os.makedirs(val_dir, exist_ok=True)

# 2. Parameter untuk split data (80% train, 20% val)
split_ratio = 0.8
np.random.seed(42)  # Untuk hasil yang konsisten

# 3. Iterasi melalui dataset, crop ikan, dan simpan
image_count = 0
problem_count = 0  # Untuk menghitung gambar yang bermasalah

for idx, sample in enumerate(dataset):
    if idx % 100 == 0:  # Progress update setiap 100 gambar
        print(f"  Memproses gambar ke-{idx}...")
    
    try:
        # Buka gambar asli
        img = Image.open(sample.filepath).convert("RGB")
        img_width, img_height = img.size
        
        # Loop untuk setiap deteksi 'Fish' dalam gambar ini
        for detection in sample.detections.detections:
            if detection.label == "Fish":
                # 1. Konversi koordinat bounding box (normalized -> pixel)
                x_min_norm, y_min_norm, width_norm, height_norm = detection.bounding_box
                x_min = int(x_min_norm * img_width)
                y_min = int(y_min_norm * img_height)
                x_max = int((x_min_norm + width_norm) * img_width)
                y_max = int((y_min_norm + height_norm) * img_height)
                
                # 2. Pastikan koordinat valid dan area tidak terlalu kecil
                #    Juga pastikan koordinat tidak keluar dari batas gambar
                if (x_max - x_min) > 20 and (y_max - y_min) > 20:
                    # Pastikan koordinat dalam batas gambar
                    x_min = max(0, x_min)
                    y_min = max(0, y_min)
                    x_max = min(img_width, x_max)
                    y_max = min(img_height, y_max)
                    
                    # Crop gambar
                    cropped_img = img.crop((x_min, y_min, x_max, y_max))
                    
                    # 3. Tentukan folder penyimpanan (train atau val)
                    save_dir = train_dir if np.random.random() < split_ratio else val_dir
                    
                    # 4. Simpan gambar yang sudah dipotong
                    # Gunakan nama yang lebih informatif
                    base_name = os.path.basename(sample.filepath).split('.')[0]
                    filename = f"{base_name}_fish_{image_count:06d}.jpg"
                    save_path = os.path.join(save_dir, filename)
                    cropped_img.save(save_path)
                    image_count += 1
    
    except Exception as e:
        problem_count += 1
        # Catat error tapi lanjutkan proses
        if problem_count <= 5:  # Hanya tampilkan 5 error pertama
            print(f"    ⚠️  Error memproses {sample.filepath}: {str(e)[:50]}...")

print("\n" + "="*60)
print("PROSES CROPPING SELESAI!")
print("="*60)
print(f"✅ Total {image_count} gambar ikan berhasil disimpan.")
print(f"⚠️  Terjadi {problem_count} masalah saat memproses beberapa gambar.")
print(f"\n📁 Struktur Folder Hasil:")
print(f"   - Data Latih (Train): {train_dir}")
print(f"     -> {len(os.listdir(train_dir))} file")
print(f"   - Data Validasi (Val): {val_dir}")
print(f"     -> {len(os.listdir(val_dir))} file")

# ---- BAGIAN 4: VERIFIKASI HASIL ----
print("\n" + "="*60)
print("VERIFIKASI HASIL CROPPING")
print("="*60)

# Cek beberapa gambar hasil crop dari folder train
import matplotlib.pyplot as plt

train_files = os.listdir(train_dir)[:5]  # Ambil 5 file pertama

if train_files and image_count > 0:
    print(f"Menampilkan 3 contoh hasil crop:")
    
    fig, axes = plt.subplots(1, 3, figsize=(12, 4))
    for i, filename in enumerate(train_files[:3]):
        try:
            img_path = os.path.join(train_dir, filename)
            img_display = Image.open(img_path)
            axes[i].imshow(img_display)
            axes[i].set_title(f"Contoh {i+1}\n{filename[:15]}...")
            axes[i].axis('off')
            img_display.close()
        except:
            axes[i].text(0.5, 0.5, "Gagal memuat", 
                        ha='center', va='center', transform=axes[i].transAxes)
            axes[i].set_title(f"Contoh {i+1} (Error)")
            axes[i].axis('off')
    
    plt.tight_layout()
    plt.show()
    
    # Tampilkan informasi statistik
    print(f"\n📊 Statistik Ukuran File (dari folder train):")
    file_sizes = []
    for filename in train_files[:20]:  # Cek 20 file pertama
        filepath = os.path.join(train_dir, filename)
        file_sizes.append(os.path.getsize(filepath) / 1024)  # Ukuran dalam KB
    
    if file_sizes:
        print(f"   - Rata-rata: {np.mean(file_sizes):.1f} KB")
        print(f"   - Minimal: {np.min(file_sizes):.1f} KB")
        print(f"   - Maksimal: {np.max(file_sizes):.1f} KB")
else:
    print("⚠️  Tidak ada file di folder train. Proses cropping mungkin gagal.")